In [1]:
!pip install openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 16.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [19]:
COLLEGE_PATH = "College Results View 2021 Data Dump for Export.xlsx - College Results View 2021 Data .csv"
AFFORD_PATH = "Affordability Gap Data AY2022-23 2.17.25.xlsx - Affordability_latest_02-17-25 1.csv"

college = pd.read_csv(COLLEGE_PATH)
afford = pd.read_csv(AFFORD_PATH)

college = college.rename(columns={
     "UNIQUE_IDENTIFICATION_NUMBER_OF_THE_INSTITUTION": "Unit ID"
})

df = college.merge(afford, on="Unit ID", how="inner")
df["Institution Name"] = df["Institution Name_x"].fillna(df["Institution Name_y"])

rename_map = {
    # Outcomes & graduation
    "Bachelor's Degree Graduation Rate Bachelor Degree Within 6 Years - Total": "grad_overall",
    "Bachelor's Degree Graduation Rate Within 6 Years - Black, Non-Latino": "grad_black",
    "Bachelor's Degree Graduation Rate Within 6 Years - Latino": "grad_latino",

    # Debt & earnings
    "Median Debt of Completers": "median_debt",
    "Median Earnings of Students Working and Not Enrolled 10 Years After Entry": "median_earn_10yr",

    # Affordability (from Affordability Gap file)
    "Net Price": "net_price",
    "Affordability Gap (net price minus income earned working 10 hrs at min wage)": "aff_gap_student",
    "Student Parent Affordability Gap: Center-Based Care": "aff_gap_parent_center",
    "Student Parent Affordability Gap: Home-Based Care": "aff_gap_parent_home",

    # Context
    "State Abbreviation": "state",
    "Sector Name": "sector_name",
    "Sector": "sector_code",

    # Demographics
    "Percent of Black or African American Undergraduates": "pct_black_ug",
    "Percent of Latino Undergraduates": "pct_latino_ug",
    "Percent of First-Time, Full-Time Undergraduates Awarded Pell Grants": "pct_pell_ftft",
}

df = df.rename(columns=rename_map)

def scale(series: pd.Series) -> pd.Series: 
    if series.isna().all(): 
        return pd.Series(0.5, index=series.index)
    s_min = series.min()
    s_max = series.max()
    if pd.isna(s_min) or pd.isna(s_max) or s_max == s_min: 
        return pd.Series(0.5, index=series.index)
    return (series - s_min) / (s_max - s_min)

group_cols = ["state", "sector_name"]
core_numeric = [
    "median_earn_10yr", 
    "median_debt", 
    "net_price", 
    "aff_gap_student", 
    "aff_gap_parent_center", 
    "aff_gap_parent_home", 
    "grad_overall", 
    "grad_black", 
    "grad_latino",
]

for col in core_numeric: 
    if col in df.columns: 
        df[col] = df.groupby(group_cols)[col].transform(
            lambda s: s.fillna(s.median())
        )
        df[col] = df[col].fillna(df[col].median())

critical = ["median_earn_10yr", "median_debt", "net_price", "aff_gap_student"]
df = df.dropna(subset=critical).copy()
df["aff_gap_parent"] = df[["aff_gap_parent_center", "aff_gap_parent_home"]].mean(axis=1, skipna=True)

ModuleNotFoundError: No module named 'openpyxl'

In [22]:
df["earnings_score"] = scale(df["median_earn_10yr"])
df["debt_score"] = 1 - scale(df["median_debt"])
df["net_price_score"] = 1 - scale(df["net_price"])
df["aff_gap_score"] = 1 - scale(df["aff_gap_student"])
df["aff_gap_parent_score"] = 1 - scale(df["aff_gap_parent"])
df["grad_score"] = scale(df["grad_overall"])

equity_components = []

for sub in ["black", "latino"]:
    col_sub = f"grad_{sub}"
    if col_sub in df.columns:
        gap_col = f"grad_gap_{sub}"
        
        gap = (df["grad_overall"] - df[col_sub]).clip(lower=0)
        df[gap_col] = gap
        
        df[f"{gap_col}_scaled"] = scale(gap)
        
        df[f"equity_{sub}"] = 1 - df[f"{gap_col}_scaled"]

        equity_components.append(f"equity_{sub}")

df["equity_score"] = df[equity_components].mean(axis=1)


NameError: name 'scale' is not defined

In [25]:
df["equity_score_filled"] = df["equity_score"].fillna(df["equity_score"].median())

weights = {
    "earnings_score": 0.30, 
    "debt_score": 0.20, 
    "net_price_score": 0.15, 
    "aff_gap_score": 0.15, 
    "aff_gap_parent_score": 0.05, 
    "equity_score_filled": 0.15,
}

wsum = sum(weights.values())
weights = {k: v / wsum for k, v in weights.items()}

df["value_equity_score"] = 0.0

for k, w in weights.items():
    if k not in df.columns:
        raise KeyError(f"Missing required score column: {k}")
    df["value_equity_score"] += w * df[k]

KeyError: 'equity_score'

In [8]:
export_cols = [
    "Unit ID",
    "Institution Name",
    "state",
    "sector_name",
    "sector_code",

    # raw metrics
    "median_earn_10yr",
    "median_debt",
    "net_price",
    "aff_gap_student",
    "aff_gap_parent",
    "aff_gap_parent_center",
    "aff_gap_parent_home",
    "grad_overall",
    "grad_black",
    "grad_latino",
    "pct_black_ug",
    "pct_latino_ug",
    "pct_pell_ftft",

    # scores
    "earnings_score",
    "debt_score",
    "net_price_score",
    "aff_gap_score",
    "aff_gap_parent_score",
    "grad_score",
    "equity_black",
    "equity_latino",
    "equity_score",
    "value_equity_score",
]

tableau_df = df[export_cols].copy()
tableau_df.to_csv("tableau_ready_colleges.csv", index=False)

print("Exported tableau_ready_colleges.csv with", len(tableau_df), "rows.")


✅ Exported tableau_ready_colleges.csv with 21001 rows.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=172841fe-d13b-4ca9-99a4-5716bfc8b95a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>